In [2]:
import scispacy
import json
import spacy
from sklearn.utils import shuffle
import random

## Data

In [1]:
types = ('CORONAVIRUS', 'IMMUNE_RESPONSE')

In [3]:
X, y = [], []

In [4]:
with open('./SS-NER-prune.json') as file:
    for line in file:
        d = json.loads(line)
        X.append(d)

In [5]:
test_X = X[:int(len(X)*(0.2))]
# test_y = y[:10]

## Spacy

### en_ner_bc5cdr

In [6]:
bc5cdr = spacy.load("en_ner_bc5cdr_md")

In [6]:
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [98]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

In [90]:

scorer = Scorer(bc5cdr)

In [7]:
from collections import defaultdict

In [8]:
ner = bc5cdr.get_pipe('ner')

In [9]:
ner.labels

('CHEMICAL', 'DISEASE')

In [10]:
maps = {'CHEMICAL': 'IMMUNE_RESPONSE', 'DISEASE': 'CORONAVIRUS'}

In [ ]:
pred_y = defaultdict(list)
ground_y = defaultdict(list)
i = 0
for d in test_X:
    if i % 100 == 0: print(i)
    doc = bc5cdr(d['body'])
    tmp = defaultdict(list)
    for e in doc.ents:
        tmp[maps[e.label_]].append(e.text)
    for key, val in tmp.items():
        pred_y[key].extend(val)
    ground = defaultdict(list)
    for e in d['entities']:
        if e['type'] in types:
            ground[e['type']].append(e['text'])
    for key, val in ground.items():
        ground_y[key].extend(val)
    i += 1

In [13]:
from collections import Counter

In [14]:
def f1(y_true, y_pred):
    i = list((Counter(y_true) & Counter(y_pred)).elements())
    # i = set(y_true).intersection(y_pred)
    recall = len(i) / len(y_true)
    precision = len(i) / len(y_pred)
    if recall + precision == 0:
        return 0
    else:
        return 2 * (precision * recall) / (precision + recall)

In [15]:
def avgf1(true_y, pred_y):
    f_list = []
    for t, p in zip(true_y, pred_y):
        f_list.append(f1(t, p))
    return sum(f_list)/len(f_list)

In [16]:
total = 0
for key, val in ground_y.items():
    tmp = avgf1(val, pred_y[key])
    total += tmp
    print(key, tmp)
print('over all f1', total/len(types))

CORONAVIRUS 0.1530527883940873
IMMUNE_RESPONSE 0.19816800293983186
over all f1 0.17561039566695957


In [94]:
# scores = scorer.score(tryx)

In [72]:
# sm = spacy.load("en_core_sci_sm")

In [ ]:
# pred_y = defaultdict(list)
# ground_y = defaultdict(list)
# i = 0
# for d in test_X:
#     if i % 100 == 0: print(i)
#     doc = sm(d['body'])
#     tmp = defaultdict(list)
#     for e in doc.ents:
#         tmp[e.label_].append(e.text)
#     for key, val in tmp.items():
#         pred_y[key].extend(val)
#     ground = defaultdict(list)
#     for e in d['entities']:
#         if e['type'] in types:
#             ground[e['type']].append(e['text'])
#     for key, val in ground.items():
#         ground_y[key].extend(val)
#     i += 1